# Parte 1 - RAG - Retrieval-Augmented Generation

### Objetivo: Desenvolver um sistema que responde perguntas sobre um conjunto de artigos científicos locais (PDFs), usando uma abordagem de Retrieval-Augmented Generation.






#ATIVIDADE

- Altere os pontos marcados com #TODO(tópico 5 e 6)
- Carregar seus próprios artigos e datasets em PDF (tópico 2).
- Usar modelo gratuito (nossa sugestão é o llama via groq).
- Avaliar respostas automaticamente com métricas de NLP.

**Observação 01:** cada aluno deve adaptar o código a um domínio específico da sua linha de pesquisa (ex: Engenharia de software, IHC, IA, robótica, etc) e comparar a performance.


**Observação 02:** Caso necessário, faça suas alterações no código, conforme os conceitos vistos em sala de aula, para adequar ao caso específico que esteja tratando.

In [1]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.8 MB/s eta 0:00:00


# 1. Upload dos PDFs
Você carregará os PDFs que gostaria que fossem analisados.

In [2]:
from google.colab import files
uploaded = files.upload()

import os
from PyPDF2 import PdfReader

# Cria pasta para os PDFs
os.makedirs("corpus", exist_ok=True)
for fname in uploaded.keys():
    os.rename(fname, os.path.join("corpus", fname))

print("PDFs carregados:", os.listdir("corpus"))

Saving Book-Web-Semantica.pdf to Book-Web-Semantica.pdf
PDFs carregados: ['Book-Web-Semantica.pdf']


# 2. Leitura e extração do texto dos PDFs
A função abaixo irá gerar o corpus (que é uma lista de textos). Cada elemento do corpus é o texto de um PDF carregado anteriormente.

In [3]:
def load_papers(folder):
    corpus = []
    for file in os.listdir(folder):
        if file.endswith(".pdf"):
            reader = PdfReader(os.path.join(folder, file))
            text = " "
            for page in reader.pages:
                text = page.extract_text() or " "
                corpus.append(text)
    return corpus

texts = load_papers("corpus")
print(f"{len(texts)} chunks carregados.")

133 chunks carregados.


# 3. Embeddings - Criação
Usar o sentence-transformers para transformar os textos extraídos dos PDFs em embeddings. (Se colab pedir acesso, conceda)

In [4]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(texts, convert_to_tensor=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# 4. Função de Recuperação - (R)AG
### Implementar o mecanismo de busca vetorial. Aqui entra o retriever: busca semântica por similaridade de embeddings.

In [5]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def retrieve(query, texts, embeddings, top_k=2, max_chars=3000):
    """
    Recupera os textos mais relevantes limitando o tamanho total (max_chars)
    para não exceder o limite de tokens do modelo Groq.
    """
    query_emb = model.encode([query])
    scores = cosine_similarity(query_emb, embeddings)[0]
    top_indices = np.argsort(scores)[::-1][:top_k]

    results = []
    total_len = 0
    for i in top_indices:
        snippet = texts[i]
        if total_len + len(snippet) > max_chars:
            snippet = snippet[: max_chars - total_len]  # corta para caber no limite
        results.append(snippet)
        total_len += len(snippet)
        if total_len >= max_chars:
            break
    return results

# 5. Integrar com uma LLM - R(AG)

### O conteúdo recuperado é passado como contexto ao modelo llama-3.3:70b a partir do groq

In [6]:
!pip install --upgrade langchain langchain-core langchain-groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.9/469.9 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.3/208.3 kB 11.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.27
    Uninstalling langchain-0.3.27:
      Successfully uninstalled langchain-0.3.27


In [9]:
from langchain_groq import ChatGroq

#TODO sigas os passos em https://groq.com/ para pegar sua chave: Developers -> Free API key
GROQ_API_KEY = input( "Cole sua chave Groq aqui e dê <enter>: ")

client = ChatGroq(
    model="llama-3.3-70b-versatile",
    api_key=GROQ_API_KEY,
    temperature=0.2
)

def generate_answer(query, context):
    prompt = f"""
Use o contexto abaixo para responder a pergunta com precisão científica.
Contexto: {context}
Pergunta: {query}
"""
    response = client.invoke(prompt).content
    return response

Cole sua chave Groq aqui e dê <enter>: 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


# 6. Teste do modelo

In [10]:

query = "O que são ontologias?"
context = " ".join(retrieve(query, texts, embeddings))
answer = generate_answer(query, context)
print("\nResposta gerada:\n", answer)


Resposta gerada:
 Ontologias são modelos conceituais que definem um conjunto de conceitos, relações e regras para representar um domínio específico de conhecimento. Elas são utilizadas para fornecer uma estrutura comum para a representação e compartilhamento de informações, permitindo que diferentes sistemas e aplicativos possam entender e interagir com essas informações de forma consistente.

No contexto da Web Semântica, as ontologias são utilizadas para definir a estrutura e o significado dos dados, permitindo que os computadores possam entender e processar esses dados de forma mais eficaz. As ontologias podem ser utilizadas para representar conceitos como entidades, relações, propriedades e restrições, e podem ser utilizadas em uma variedade de aplicações, incluindo busca de informações, integração de dados e tomada de decisões.

No exemplo mencionado no texto, a ontologia PROV é utilizada para representar a proveniência de dados, ou seja, a origem e a história dos dados. Ela defi

# Parte 2 - Pesquisa na web - Web-based RAG ou Online RAG
Nesta seção, faremos uma prática de buscas de informações na web. O objetivo é dar ao modelo dados atualizados retirados de artigos na web. Neste exemplo, no Retrieval **(Recuperação)**, ao invés de buscarmos de um corpus de documentos ou banco de dados, buscaremos da web. O conteúdo extraído da página da web será usado para Aumentar **(Augment)** o prompt fornecido ao modelo. E por fim, o modelo usará o prompt para Gerar **(Generation)** uma resposta que será o resumo de um artigo.

Em resumo, iremos:
demonstrar um fluxo de RAG (Retrieval Augmented Generation) buscando informações na internet usando DuckDuckGo, extraindo o conteúdo de um artigo relevante e resumindo-o usando o LLM pelo Groq.

#ATIVIDADE

**Altere os pontos marcados com #TODO** no código para testar diferentes
resultados.

Por exemplo, se houver: QUANT_MAX_ARTIGOS = 5  # TODO
mude para 10, por exemplo e veja como muda a seleção de artigos.

**Escolha o artigo que será usado.**

Por padrão, search_results[0] pega apenas o primeiro. Você pode testar com outro índice para ver respostas diferentes.

**Volte à Parte 1 e repita a execução.**

Lá o código junta os resultados do retrieve() nos chunks e passa esse contexto para o LLM.

Assim, você consegue comparar como as alterações influenciam a resposta final do modelo.





## 1. Instalar bibliotecas necessárias

Instalar bibliotecas para buscar na web (DuckDuckGo) e para extrair o conteúdo de páginas web.


In [11]:
!pip install ddgs
!pip install beautifulsoup4
!pip install requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 78.2 MB/s eta 0:00:00
  Attempting uninstall: lxml
    Found existing installation: lxml 5.4.0
    Uninstalling lxml-5.4.0:
      Successfully uninstalled lxml-5.4.0


##2. Realizar busca na web

Usar a ferramenta de busca para encontrar artigos relevantes com base em uma consulta do usuário. O duckduckgo (ddgs) faz o trabalho de buscar artigos na Internet, assim como o Google.


In [12]:
from ddgs import DDGS

ddgs = DDGS()
QUANT_MAX_ARTIGOS = 5

query = "Ontology and Semantic Web"
search_results = ddgs.text(query, max_results=QUANT_MAX_ARTIGOS)

print("Resultados da busca:")
for result in search_results:
    print(f"Título: {result['title']}")
    print(f"URL: {result['href']}")
    print(f"Descrição: {result['body']}\n")

Resultados da busca:
Título: OWL Web Ontology Language Semantics and Abstract Syntax
URL: https://www.w3.org/TR/owl-semantics/
Descrição: 30 May 2003] Per a decision of the Web Ontology working group on 29 May 2003 to change the semantics for owl:intersectionOf and related resources ...

Título: Web Ontology Language (OWL) Abstract Syntax and Semantics
URL: https://www.w3.org/TR/2002/WD-owl-semantics-20021108/
Descrição: The n-triples syntax and RDFS-compatible semantics allows annotation of ontologies by using RDF properties on the document URI and annotation of ...

Título: Semantic Web Programming and Ontology Construction Courses
URL: https://ftt.co.uk/Semantic_Web_Courses.php
Descrição: Semantic Web Programming and Ontology Construction Courses ... SWEB101 Introduction to the Semantic Web - RDF, RDFS, OWL and Ontologies (5 days)

Título: Ontology and the Semantic Web PDF Download Free | 1586037293
URL: https://ebooks-it.org/1586037293-ebook.htm
Descrição: ... Semantic Web works te

##3. Extrair conteúdo do artigo

Acessar a URL do artigo retornado pela busca e extrair o texto principal.


In [24]:
import requests
from bs4 import BeautifulSoup
import random

article_text = None
if not search_results:
    print("Nenhuma URL encontrada para extração.")
else:

    #Escolher um artigo aleatoriamente
    indices = list(range(QUANT_MAX_ARTIGOS))
    #Embaralha a ordem dos índices
    random.shuffle(indices)

    #Tenta acessar os artigos em ordem aleatória até encontrar um que possa ser baixado com sucesso.
    for idx in indices:
        article_url = search_results[idx]['href']


    try:
        response = requests.get(article_url, timeout=10) # Adicionado timeout para evitar travamentos
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            # Tentativa de encontrar o texto principal. Isso pode precisar de ajustes
            # dependendo da estrutura HTML dos sites.
            article_text = ""
            paragraphs = soup.find_all('p')
            for p in paragraphs:
                article_text += p.get_text() + "\n"

            if article_text:
                print(f"Conteúdo do artigo extraído da URL: {article_url}")
                print("Primeiros 500 caracteres do texto extraído:")
                print(article_text[:500])
            else:
                print(f"Não foi possível extrair texto principal da URL: {article_url}")

        else:
            print(f"Erro ao acessar a URL {article_url}. Código de status: {response.status_code}")
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar a URL {article_url}: {e}")

if article_text:
    print(len(article_text))
else:
    #Imprime esta mensagem caso dê erro na extração
    print("Nenhum artigo extraído.")

Conteúdo do artigo extraído da URL: https://ftt.co.uk/Semantic_Web_Courses.php
Primeiros 500 caracteres do texto extraído:
Standard and Advanced Technical Training, Consultancy and Mentoring

 Semantic Web Programming and Ontology Construction Courses  

At present, information on the web is oriented towards human consumption. 
Computers are presently mainly used as means for storing and conveying that information. 
Humans understand the meaning of the information they are using, 
whereas, for a computer that information is just a large collection of symbols. 
In order for computers to be able to use web inform
1993


##4. Quebrando os chunks

Quebrar em chunks o texto extraído para utilizar com contexto. Utilizando outra abordagem.


In [25]:
!pip install langchain langchain-text-splitters

In [26]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Configura o Text Splitter
# Instancia o separador de texto com as suas especificações
text_splitter = RecursiveCharacterTextSplitter(
    # Tamanho máximo de cada chunk em caracteres
    chunk_size=1500,
    # Tamanho de sobreposição entre chunks.
    # Isso ajuda a manter o contexto entre os chunks adjacentes.
    chunk_overlap=250,
    # Separadores que o splitter tentará usar, em ordem:
    # 1. Parágrafos (\n\n)
    # 2. Novas linhas (\n)
    # 3. Espaços (' ')
    # 4. Caracteres vazios ('')
    separators=["\n\n", "\n", " ", ""],
    length_function=len # Função usada para medir o tamanho (len para caracteres)
)

# 3. Quebrar o texto
chunks = text_splitter.create_documents([article_text])

# 4. Imprimir os resultados para verificação

print(f"Número total de Chunks criados: **{len(chunks)}**\n")
print("-" * 50)

# Itera sobre os chunks (objetos Document do LangChain)
for i, chunk in enumerate(chunks):
    content = chunk.page_content
    print(f"*** CHUNK {i+1} (Tamanho: {len(content)} caracteres) ***")
    print(content)
    print("-" * 50)

Número total de Chunks criados: **2**

--------------------------------------------------
*** CHUNK 1 (Tamanho: 1226 caracteres) ***
Standard and Advanced Technical Training, Consultancy and Mentoring

 Semantic Web Programming and Ontology Construction Courses  

At present, information on the web is oriented towards human consumption. 
Computers are presently mainly used as means for storing and conveying that information. 
Humans understand the meaning of the information they are using, 
whereas, for a computer that information is just a large collection of symbols. 
In order for computers to be able to use web information they need some mechanism 
for understanding its meaning. 
This is the classical issue of information vs. knowledge. 
Knowledge permits reasoning and planning. 
As the web is a network oriented medium so, knowledge on the web can be 
envisaged as a net - in effect, a semantic network. 


Web pages use markup in the form of HTML/XHTML. XHTML documents are based on X